# OpenVaccine-COVID-19-mRNA-Vaccine-Degradation-Prediction-baseline-lightGBM

## debug

In [264]:
debug = True

## import libraries

In [265]:
import pandas as pd
import numpy as np
import copy
from sklearn.metrics import mean_squared_error
import json
import re

import lightgbm as lgb
import category_encoders as ce

## load data

In [266]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)
submission = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')

## preprocess

In [267]:
def preprocess(train: pd.DataFrame, test: pd.DataFrame):
    #test_cols = test.columns
    
    df = pd.concat([train, test])
    df["G"] = df["sequence"].map(lambda x: x.count("G"))
    df["A"] = df["sequence"].map(lambda x: x.count("A"))
    df["C"] = df["sequence"].map(lambda x: x.count("C"))
    df["U"] = df["sequence"].map(lambda x: x.count("U"))
    
    df["A_U"] = 0
    df["G_C"] = 0
    for i in range(len(df["structure"])):
        s_list = [m.start() for m in re.finditer("\(", df["structure"].iloc[i])]
        t_list = [m.start() for m in re.finditer("\)", df["structure"].iloc[i])]
        pair_list = []
        for j in s_list:
            pair_list.append(df["sequence"].iloc[i][j])
        for k in t_list:
            pair_list.append(df["sequence"].iloc[i][k])
        df["A_U"].iloc[i] = pair_list.count("A")
        df["G_C"].iloc[i] = pair_list.count("G")
    
    df["S"] = df["predicted_loop_type"].map(lambda x: x.count("S"))
    df["M"] = df["predicted_loop_type"].map(lambda x: x.count("M"))
    df["I"] = df["predicted_loop_type"].map(lambda x: x.count("I"))
    df["B"] = df["predicted_loop_type"].map(lambda x: x.count("B"))
    df["H"] = df["predicted_loop_type"].map(lambda x: x.count("H"))
    df["E"] = df["predicted_loop_type"].map(lambda x: x.count("E"))
    df["X"] = df["predicted_loop_type"].map(lambda x: x.count("X"))
    
    for n in range(107):
        df[f'sequence_{n}'] = df['sequence'].map(lambda x: x[n])
        df[f'structure_{n}'] = df['structure'].map(lambda x: x[n])
        df[f'predicted_loop_type_{n}'] = df['predicted_loop_type'].apply(lambda x: x[n])
    
    SEQUENCE_COLS = [c for c in df.columns if 'sequence_' in c]
    STRUCTURE_COLS = [c for c in df.columns if 'structure_' in c]
    PREDICTED_LOOP_COLS = [c for c in df.columns if 'predicted_loop_type_' in c]
    OTHERS = ["predicted_loop_type"]
    object_cols = SEQUENCE_COLS + STRUCTURE_COLS + PREDICTED_LOOP_COLS + OTHERS
    #object_cols = ["predicted_loop_type"]
    ce_oe = ce.OrdinalEncoder(cols=object_cols, handle_unknown='impute')
    df = ce_oe.fit_transform(df)
    
    #df = df.drop(columns=["sequence", "structure"])
    
    train = df.iloc[:len(train), :]
    test = df.iloc[len(train):, :]
    
    
    train = train.drop(columns=["sequence", "structure", "index"])
    test = test.drop(columns=["sequence", "structure", "index"])
    
    
    return train, test
    
    

In [268]:
train, test = preprocess(train, test)

/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [269]:
def preprocess_train(train: pd.DataFrame):
    train['reactivity_error'] = train['reactivity_error'].map(lambda x: np.mean(x))
    train['deg_error_Mg_pH10'] = train['deg_error_Mg_pH10'].map(lambda x: np.mean(x))
    train['deg_error_Mg_50C'] = train['deg_error_Mg_50C'].map(lambda x: np.mean(x))
    
    required_mean = train['reactivity_error'][train['reactivity_error'] > 1].mean()
    train['reactivity_error'][train['reactivity_error'] <= 1] = required_mean

    required_mean = train['deg_error_Mg_pH10'][train['deg_error_Mg_pH10'] > 1].mean()
    train['deg_error_Mg_pH10'][train['deg_error_Mg_pH10'] <= 1] = required_mean

    required_mean = train['deg_error_Mg_50C'][train['deg_error_Mg_50C'] > 1].mean()
    train['deg_error_Mg_50C'][train['deg_error_Mg_50C'] <= 1] = required_mean
    
    train['reactivity'] = train['reactivity'].apply(lambda x: np.mean(x))# + train['reactivity_error']
    train['deg_Mg_pH10'] = train['deg_Mg_pH10'].apply(lambda x: np.mean(x))# + train['deg_error_Mg_pH10']
    train['deg_Mg_50C'] = train['deg_Mg_50C'].apply(lambda x: np.mean(x))# + train['deg_error_Mg_50C']
    
    return train

In [270]:
train = preprocess_train(train)

/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## train models and prediction

In [271]:
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold
FOLD_NUM = 5
kf = KFold(n_splits=5, shuffle=True, random_state=0)
target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']
oof = train[target_cols].copy()
oof_test = test.copy()
num_round = 10000

In [272]:
SEQUENCE_COLS = [c for c in train.columns if 'sequence_' in c]
STRUCTURE_COLS = [c for c in train.columns if 'structure_' in c]
PREDICTED_LOOP_COLS = [c for c in train.columns if 'predicted_loop_type_' in c]
OTHERS = ["predicted_loop_type"]
object_cols = SEQUENCE_COLS + STRUCTURE_COLS + PREDICTED_LOOP_COLS + OTHERS

In [273]:
oof

,reactivity,deg_Mg_pH10,deg_Mg_50C
0,0.502631,0.559628,0.538540
1,0.411401,0.698354,0.930103
2,0.433966,0.578362,0.570284
3,0.329059,0.826485,1.221297
4,0.282204,0.469587,0.393072
...,...,...,...
2395,0.459390,0.511384,0.539138
2396,0.403482,0.315901,0.492297
2397,0.223107,0.354449,0.235294
2398,0.358229,0.366354,0.261793


In [274]:
params = {
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
#'lambda_l1': 0.001, 
#'lambda_l2': 0.001,
'num_leaves': 50, 
'max_depth': 6,
#'feature_fraction': 0.4,
#'subsample': 0.4, 
#'min_child_samples': 10,
'learning_rate': 0.01,
#'num_iterations': 100, #700
'early_stopping_rounds': 100,
'random_state': 42}

In [275]:
cols = ["signal_to_noise", "SN_filter", "seq_scored", "reactivity_error", "deg_error_Mg_pH10", "deg_error_pH10", "deg_error_Mg_50C", 
        "deg_error_50C", "reactivity", "deg_Mg_pH10", "deg_pH10", "deg_Mg_50C", "deg_50C"]

In [276]:
test = test.drop(columns=cols)

In [277]:
def run_lgbm(target_col: str):
    
    X_train = train.drop(["id"], axis=1)
    X_train = X_train.drop(columns=cols)
    y_train = oof[target_col]
    X_test = test.drop(["id"], axis=1)
    print(X_train.shape, X_test.shape)

    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))

    for fold_id, (train_index, valid_index) in enumerate(kf.split(X_train)):
        X_tr = X_train.loc[train_index, :]
        X_val = X_train.loc[valid_index, :]
        y_tr = y_train[train_index]
        y_val = y_train[valid_index]

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=object_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=object_cols)

        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=10,
                          num_boost_round=1000,
                          early_stopping_rounds=10)


        oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
        
        
        y_pred = model.predict(X_test, num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)

    return oof_train, sum(y_preds) / len(y_preds)

In [278]:
for col in train.columns:
    if train[col].dtypes == "object":
        print(col)

id
deg_error_pH10
deg_error_50C
deg_pH10
deg_50C


In [279]:
train.shape

(2400, 350)

In [280]:
test.shape

(3634, 337)

In [281]:
for target_col in target_cols:
    _oof, _preds = run_lgbm(target_col)
    oof[target_col] = _oof
    oof_test[target_col] = _preds

(2400, 336) (3634, 336)
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001052 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1603
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 217
[LightGBM] [Info] Start training from score 0.373099
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[20]	training's rmse: 0.111331	valid_1's rmse: 0.104739
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[130]	training's rmse: 0.0860925	valid_1's rmse: 0.0836575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	training's rmse: 0.0746911	valid_1's rmse: 0.0782818
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[110]	training's rmse: 0.10464	valid_1's rmse: 0.126965
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[220]	training's rmse: 0.0885376	valid_1's rmse: 0.120301
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[340]	training's rmse: 0.0797069	valid_1's rmse: 0.119546
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[460]	training's rmse: 0.0731081	valid_1's rmse: 0.118287
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[570]	training's rmse: 0.0683913	valid_1's rmse: 0.117627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[580]	training's rmse: 0.068043	valid_1's 

In [282]:
scores = []
for target_col in target_cols:
    scores.append(np.sqrt(mean_squared_error(train[target_col] , oof[target_col])))
print(np.mean(scores))

0.09263756100737154


In [290]:
submission

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,id
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0,id_00073f8be
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0,id_00073f8be
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0,id_00073f8be
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0,id_00073f8be
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0,id_00073f8be
...,...,...,...,...,...,...,...
457948,id_ffda94f24_125,0.0,0.0,0.0,0.0,0.0,id_ffda94f24
457949,id_ffda94f24_126,0.0,0.0,0.0,0.0,0.0,id_ffda94f24
457950,id_ffda94f24_127,0.0,0.0,0.0,0.0,0.0,id_ffda94f24
457951,id_ffda94f24_128,0.0,0.0,0.0,0.0,0.0,id_ffda94f24


In [291]:
submission['id'] = 'id_' + submission['id_seqpos'].str.split('_', expand=True)[1]

# Merge my predicted average values
ss_new = submission.drop(['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C'], axis=1).merge(oof_test[['id', 'reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']], on='id', validate='m:1')

In [292]:
submission

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,id
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0,id_00073f8be
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0,id_00073f8be
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0,id_00073f8be
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0,id_00073f8be
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0,id_00073f8be
...,...,...,...,...,...,...,...
457948,id_ffda94f24_125,0.0,0.0,0.0,0.0,0.0,id_ffda94f24
457949,id_ffda94f24_126,0.0,0.0,0.0,0.0,0.0,id_ffda94f24
457950,id_ffda94f24_127,0.0,0.0,0.0,0.0,0.0,id_ffda94f24
457951,id_ffda94f24_128,0.0,0.0,0.0,0.0,0.0,id_ffda94f24


In [295]:
if debug == True:
    ss_new[submission.columns].to_csv("submission.csv", index=False)
    ss_new.head()

## optuna

In [ ]:
def objective(trial):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        #'metric': 'rmse',
        'verbosity': -1,
        "seed":42,
        "learning_rate":trial.suggest_loguniform('learning_rate', 0.005, 0.03),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }
    FOLD_NUM = 5
    mskf = MultilabelStratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=42)
    scores = []
    feature_importance_df = pd.DataFrame()

    pred_cv = np.zeros(len(test.index))
    num_round = 10000

    


    for i, (tdx, vdx) in enumerate(kf.split(train_X[selected], train_y)):
        print(f'Fold : {i}')
        X_train, X_valid, y_train, y_valid = train_X[selected].iloc[tdx], train_X[selected].iloc[vdx], train_y.values[tdx], train_y.values[vdx]
        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=object_cols)
        lgb_valid = lgb.Dataset(X_valid, y_valid, categorical_feature=object_cols)
        model = lgb.train(params, lgb_train, num_boost_round=num_round,
                      valid_names=["train", "valid"], valid_sets=[lgb_train, lgb_valid],
                      early_stopping_rounds=100, verbose_eval=10000)

        va_pred = model.predict(X_valid)
        va_pred[va_pred<0] = 0
        score_ = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(va_pred)))
        scores.append(score_)

    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
# 結果の確認
print('Best trial:')
light_trial = study.best_trial

print('  Value: {}'.format(light_trial.value))

print('  Params: ')

In [ ]:
with open("lightgbmparams.txt", "w") as file:
    for key, value in light_trial.params.items():
       print('    "{}": {},'.format(key, value))
       file.write('"{}": {},'.format(key, value))